The Jupyter Notebook introduces Python classes designed to analyze frequencies of classes of invariants sharing similar components. It implements `Permutation` and `PermutationRep` classes enabling operations such as generating inverses, determining Black-White or Black-White-Centre color representations, and comparing these classes based on symmetries within their components. The analysis within the A8 algebra demonstrates that classes with matching frequencies exhibit relationships via Black-White inversion and inversion, explaining the occurrence of doublets and quadruplets. In E8, some doublet frequencies are explained by inversion, while the rest remain unclear. D8 displays explanations for some doublets and quadruplets through inversion and 6<->7 symmetries, hinting at further identifiable symmetries.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from numpy.random import choice
from collections import Counter
from copy import deepcopy

from tqdm import tqdm
import time

In [2]:
# Class to work with permutation and check inversion, Black-White inversion

# Class implementing individual permutations and basic operations with them
class Permutation:
  # initialise from list
  def __init__(self, permList=None):
    self.perm = []

    if permList==None:
      return

    if not isinstance(permList, list):
      raise ValueError(f"Error: Input permutation is of the wrong type. Needed List but received {type(permList)}.")

    if not len(permList) == 8:
      raise ValueError(f"Error: incorrect list size! Received object of length {len(permList)} instead of 8")

    self.perm = list(permList)

  # define output as a text
  def __str__(self):
    return f"{self.perm}"

  # define equality
  def __eq__(self, other):
    if not isinstance(other, Permutation):
        # don't attempt to compare against unrelated types
        return NotImplemented

    return self.perm == other.perm

  # returns permutation as a list
  def toList(self):
    return self.perm

  # generate inverse permutation: reverse order of roots/simple roots in a permutation
  def inverse(self):
    return Permutation(list(reversed(self.perm)))

  # output string of Balck-White coloring of a permutation
  def BWrep(self,algebraName):
    BWrepList = []

    match algebraName:
      case "A8":
        for elem in self.perm:
          if elem % 2 == 0:
            BWrepList.append('W')
          else:
            BWrepList.append('B')
        return BWrepList
      case "E8":
        for elem in self.perm:
          if elem % 2 == 0:
            BWrepList.append('W')
          else:
            BWrepList.append('B')
        return BWrepList
      case "D8":
        for elem in self.perm:
          if elem==0 or elem==2 or elem==4 or elem==6 or elem==7:
            BWrepList.append('W')
          else:
            BWrepList.append('B')
        return BWrepList
      case _:
        raise ValueError(f"Error: incorrect algebra name! Given \"{algebraName}\" instead of \"E8\", \"D8\" or \"A8\".")

  # output BLack-White inversed coloring of a permutation
  def BWrepInv(self,algebraName):
    BWrepList = []

    match algebraName:
      case "A8":
        for elem in self.perm:
          if elem % 2 == 0:
            BWrepList.append('B')
          else:
            BWrepList.append('W')
        return BWrepList
      case "E8":
        for elem in self.perm:
          if elem % 2 == 0:
            BWrepList.append('B')
          else:
            BWrepList.append('W')
        return BWrepList
      case "D8":
        for elem in self.perm:
          if elem==0 or elem==2 or elem==4 or elem==6 or elem==7:
            BWrepList.append('B')
          else:
            BWrepList.append('W')
        return BWrepList
      case _:
        raise ValueError(f"Error: incorrect algebra name! Given \"{algebraName}\" instead of \"E8\", \"D8\" or \"A8\".")

  # return permutation with roots 6 and 7 swapped, used to check 6<->7 symmetry
  def Swap67(self,algebraName):
    BWrepList = []

    if algebraName != "D8":
      raise ValueError(f"Error: incorrect algebra! Given \"{algebraName}\" instead of \"D8\" .")

    def SixSevSwap(a):
      if a==7:
        return 6
      elif a==6:
        return 7
      else:
        return a

    return Permutation([SixSevSwap(elem) for elem in self.perm])


  # output Black-White-Centre coloring of a permutation, Centre - roots 6 and 7
  def BWCrep(self,algebraName):
    BWrepList = []

    if  algebraName=="D8":
      for elem in self.perm:
        if elem==6 or elem==7:
          BWrepList.append('C')
        elif elem==0 or elem==2 or elem==4:
          BWrepList.append('W')
        else:
          BWrepList.append('B')
      return BWrepList
    else:
        raise ValueError(f"Error: incorrect algebra! Given \"{algebraName}\" instead of \"D8\".")

  # output Black-White-Centre inversed coloring of a permutation, Centre - roots 6 and 7. Only Black and White are inversed
  def BWCrepInv(self,algebraName):
    BWrepList = []

    if  algebraName=="D8":
      for elem in self.perm:
        if elem==6 or elem==7:
          BWrepList.append('C')
        elif elem==0 or elem==2 or elem==4:
          BWrepList.append('B')
        else:
          BWrepList.append('W')
      return BWrepList
    else:
        raise ValueError(f"Error: incorrect algebra! Given \"{algebraName}\" instead of \"D8\".")


# -------------------------------------------------------------------------------------------------------------

# Class implemening 'classes of permutations with the corresponding coefficient vectors being identical' = 'set of permutations which have the same coefficient vectors'. Call it permutation representation (Perm Rep)
class PermutationRep:
  # initilise Perm Rep
  def __init__(self, algebraName, perm=None):
    self.algebraName = algebraName
    self.permReps = []

    if perm == None:
      return

    if isinstance(perm, list):
      if len(perm) == 8:
        self.permReps.append(Permutation(perm))
        return
      else:
        raise ValueError(f"Error: incorrect list size! Received object of length {len(perm)} instead of 8")

    if isinstance(perm, Permutation):
      self.permReps.append(perm)
      return

    raise ValueError(f"Incorrect datatype: {type(perm)}")

  # define output as a text
  def __str__(self):
    return f"{[str(elem) for elem in self.permReps]} in {self.algebraName}"

  # define equality
  def __eq__(self, other):
        if not isinstance(other, PermutationRep):
            # don't attempt to compare against unrelated types
            raise ValueError(f"Error: Second argunment has incorrect datatype: input is {type(other)} but should be PermutationRep")

        if not self.algebraName == other.algebraName:
          # don't attempt to compare between elements for different algebras
            raise ValueError(f"Error: Elements belong to different algebras: {type(self.algebraName)} and {type(other.algebraName)}")

        for perm1 in self.permReps:
          for perm2 in other.permReps:
            if perm1 == perm2:
              return True

        return False

  # add another instance to Perm Rep
  def addPermRep(self, perm):
    if isinstance(perm, list):
      if len(perm) == 8:
        self.permReps.append(Permutation(perm))
        return
      else:
        raise ValueError(f"Error: incorrect list size! Received object of length {len(perm)} instead of 8")

    if isinstance(perm, Permutation):
      self.permReps.append(perm)
      return

    raise ValueError(f"Error: Incorrect datatype: {type(perm)}")

  # output one representative of the Perm Rep
  def representative(self):
    return self.permReps[0]

  # check if two permReps are inverse of each other
  def isInverse(self, other):
        if not isinstance(other, PermutationRep):
            # don't attempt to compare against unrelated types
            raise ValueError(f"Error: Second argunment has incorrect datatype: {type(other)}")

        flag = True

        for perm1 in self.permReps: # for each element in the first Perm Rep we should find an inverse in the second Perm Rep
          flag2 = False
          for perm2 in other.permReps:
            if perm1.inverse() == perm2:
              flag2 = True
          flag = bool(flag*flag2)

        return flag

  # check that two permReps are Black-White inversed
  def isBWInverse(self, other):
    if not isinstance(other, PermutationRep):
      # don't attempt to compare against unrelated types
      raise ValueError(f"Error: Second argunment has incorrect datatype: {type(other)}")

    # Make lists of permutations hashable (to further sort and compare them)
    first_tuple_list = [tuple(lst.BWrep(self.algebraName)) for lst in self.permReps]
    secnd_tuple_list = [tuple(lst.BWrepInv(other.algebraName)) for lst in other.permReps]

    first_tuple_list.sort()
    secnd_tuple_list.sort()

    if first_tuple_list==secnd_tuple_list:
      return True
    return False


  # check that two permReps are Black-White identical
  def isBWIdentical(self, other):
    if not isinstance(other, PermutationRep):
      # don't attempt to compare against unrelated types
      raise ValueError(f"Error: Second argunment has incorrect datatype: {type(other)}")

    # Make lists of permutations hashable (to further compare them)
    first_tuple_list = [tuple(lst.BWrep(self.algebraName)) for lst in self.permReps]
    secnd_tuple_list = [tuple(lst.BWrep(other.algebraName)) for lst in other.permReps]

    first_tuple_list.sort()
    secnd_tuple_list.sort()

    if first_tuple_list==secnd_tuple_list:
      return True
    return False

  # check that two permReps are 6<->7 inversed
  def is67Inverse(self, other):
    if not isinstance(other, PermutationRep):
      # don't attempt to compare against unrelated types
      raise ValueError(f"Error: Second argunment has incorrect datatype: {type(other)}")

    # Make lists of permutations hashable (to further compare them)
    first_tuple_list = [tuple(lst.perm) for lst in self.permReps]
    secnd_tuple_list = [tuple(lst.Swap67(other.algebraName).perm) for lst in other.permReps]

    first_tuple_list.sort()
    secnd_tuple_list.sort()

    if first_tuple_list==secnd_tuple_list:
      return True
    return False


  # check that twp permReps are Black-White-Centre inversed
  def isBWCInverse(self, other):
    if not isinstance(other, PermutationRep):
      # don't attempt to compare against unrelated types
      raise ValueError(f"Error: Second argunment has incorrect datatype: {type(other)}")

    # Make lists of permutations hashable (to further compare them)
    first_tuple_list = [tuple(lst.BWCrep(self.algebraName)) for lst in self.permReps]
    secnd_tuple_list = [tuple(lst.BWCrepInv(other.algebraName)) for lst in other.permReps]

    first_tuple_list.sort()
    secnd_tuple_list.sort()

    if first_tuple_list==secnd_tuple_list:
      return True
    return False

  # check that twp permReps are Black-White-Centre identical
  def isBWCIdentical(self, other):
    if not isinstance(other, PermutationRep):
      # don't attempt to compare against unrelated types
      raise ValueError(f"Error: Second argunment has incorrect datatype: {type(other)}")

    # Make lists of permutations hashable (to further compare them)
    first_tuple_list = [tuple(lst.BWCrep(self.algebraName)) for lst in self.permReps]
    secnd_tuple_list = [tuple(lst.BWCrep(other.algebraName)) for lst in other.permReps]

    first_tuple_list.sort()
    secnd_tuple_list.sort()

    if first_tuple_list==secnd_tuple_list:
      return True
    return False

A8

In [4]:
# Import data -- in format
# [[(permutation order of roots in W definition), [list of invariant coefficient vectors], ...all permutations]

# Parse string into list of lists of float
def parseString(string):
    tmp = list(string[1:-2].split(", [["))
    return [tmp[0], [ [float(i) for i in list(elem.split(", "))]  for elem in list(tmp[1][:-1].split("], ["))] ]

start = time.time()

# Modified files are used: all 1/2, 3/2, 5/2 are replaced by 0.5, 1.5, 2,5
with open('ADE_SimpleRootData\A8inv_SimpleRootData_mod.txt','r') as file:
    data = [ parseString(line.rstrip()) for line in file]

end = time.time()
print(end - start)

data_size = len(data)

78.55321884155273


In [5]:
# Find class of each invariant in A8 (class = invaraints with the same components)
elemsClassId = np.zeros(data_size, dtype = int)
for idx1 in tqdm(range(len(data))):
    if elemsClassId[idx1] != 0: # check if we've assigned a class to this element already
        continue
    elemsClassId[idx1] = idx1+1 # record that elem itself in its own class (shift classes number's by 1 during run for technical reasons)
    for idx2 in range(idx1+1,len(data)):
        if data[idx1][1]==data[idx2][1]: # identify components which are the same
            elemsClassId[idx2] = idx1+1 # record that elem idx2 is in the same class as element idx1 (shift classes number's by 1 during run)

elemsClassId = elemsClassId - 1 # shift classes number's back by 1

100%|██████████| 40320/40320 [00:36<00:00, 1116.28it/s]


In [7]:
# Collect equivalent permutation for each class
elemsClassPerms = []
for classId in set(elemsClassId):
  elemsClassPerms.append([classId,PermutationRep('A8'),0]) # [Abstract id of the class, permutation class, frequency (initialised as 0)]

  for idx in range(data_size):
    if elemsClassId[idx] == classId:
      elemsClassPerms[-1][1].addPermRep(list(eval(data[idx][0])))

print(len(elemsClassPerms))

128


In [8]:
# Create a dictionary containing pairs (class index, number of elements in class)
classes_counter = dict(Counter(elemsClassId))
print('Number of classes in A8: ', len(classes_counter))

# Sort classes by the number of elements
tempDict = classes_counter
classes_counter = {k: v for k, v in sorted(tempDict.items(), key=lambda item: item[1])}

Number of classes in A8:  128


In [9]:
# add correct frequencies to every class
for idx in range(len(elemsClassPerms)):
  elemsClassPerms[idx][2] = classes_counter[elemsClassPerms[idx][0]]

In [10]:
# go over classes with the same frequencies and see which permReps are related by inversion or BW inversed duality
for frequency in sorted(list(set(classes_counter.values()))):
  print(f"Frequency {frequency} class:")

  sameFreqReps = [] # create lest to keep same frequency permReps
  for record in elemsClassPerms:
    if record[2] == frequency:
      sameFreqReps.append(record[1]) #add permReps with the same frequency into list)

  print('There are ',len(sameFreqReps),' permReps there ')

  n = len(sameFreqReps)
  for i in range(n):
    for j in range(i+1,n):
      if i!=j:
        if sameFreqReps[i].isInverse(sameFreqReps[j]):
          print(f"Permutation classes {i} and {j} are related by inversion")
        if sameFreqReps[i].isBWInverse(sameFreqReps[j]):
          print(f"Permutation classes {i} and {j} are related by BW inversion")
        if sameFreqReps[i].isBWIdentical(sameFreqReps[j]):
          print(f"Permutation classes {i} and {j} are BW identical")

  print('-----')

  del sameFreqReps

Frequency 1 class:
There are  2  permReps there 
Permutation classes 0 and 1 are related by inversion
Permutation classes 0 and 1 are related by BW inversion
-----
Frequency 7 class:
There are  4  permReps there 
Permutation classes 0 and 1 are related by inversion
Permutation classes 0 and 3 are related by BW inversion
Permutation classes 1 and 2 are related by BW inversion
Permutation classes 2 and 3 are related by inversion
-----
Frequency 21 class:
There are  4  permReps there 
Permutation classes 0 and 2 are related by BW inversion
Permutation classes 0 and 3 are related by inversion
Permutation classes 1 and 2 are related by inversion
Permutation classes 1 and 3 are related by BW inversion
-----
Frequency 27 class:
There are  4  permReps there 
Permutation classes 0 and 1 are related by BW inversion
Permutation classes 0 and 2 are BW identical
Permutation classes 0 and 3 are related by inversion
Permutation classes 0 and 3 are related by BW inversion
Permutation classes 1 and 2 a

In [11]:
del data, elemsClassId, elemsClassPerms, classes_counter, tempDict

E8

In [12]:
start = time.time()

# Modified files are used: all 1/2, 3/2, 5/2 are replaced by 0.5, 1.5, 2,5
with open('ADE_SimpleRootData\E8inv_SimpleRootData_mod.txt','r') as file:
    data = [ parseString(line.rstrip()) for line in file]

end = time.time()
print(end - start)

data_size = len(data)

52.53839087486267


In [13]:
# Find class of each invariant in E8 (class = invaraints with the same components)
elemsClassId = np.zeros(data_size, dtype = int)
for idx1 in tqdm(range(len(data))):
    if elemsClassId[idx1] != 0: # check if we've assigned a class to this element already
        continue
    elemsClassId[idx1] = idx1+1 # record that elem itself in its own class (shift classes number's by 1 during run for technical reasons)
    for idx2 in range(idx1+1,len(data)):
        if data[idx1][1]==data[idx2][1]: # identify components which are the same
            elemsClassId[idx2] = idx1+1 # record that elem idx2 is in the same class as element idx1 (shift classes number's by 1 during run)

elemsClassId = elemsClassId - 1 # shift classes number's back by 1

100%|██████████| 40320/40320 [00:33<00:00, 1190.55it/s] 


In [14]:
# Collect equivalent permutation for each class
elemsClassPerms = []
for classId in set(elemsClassId):
  elemsClassPerms.append([classId,PermutationRep('E8'),0]) # [Abstract id of the class, permutation class, frequency (initialised as 0)]

  for idx in range(data_size):
    if elemsClassId[idx] == classId:
      elemsClassPerms[-1][1].addPermRep(list(eval(data[idx][0])))

print(len(elemsClassPerms))

128


In [15]:
# Create a dictionary containing pairs (class index, number of elements in class)
classes_counter = dict(Counter(elemsClassId))
print('Number of classes in E8: ', len(classes_counter))

# Sort classes by the number of elements
tempDict = classes_counter
classes_counter = {k: v for k, v in sorted(tempDict.items(), key=lambda item: item[1])}

Number of classes in E8:  128


In [16]:
# add correct frequencies to every class
for idx in range(len(elemsClassPerms)):
  elemsClassPerms[idx][2] = classes_counter[elemsClassPerms[idx][0]]

In [17]:
# go over classes with the same frequencies and see which permReps are related by inversion or BW inversed duality
for frequency in sorted(list(set(classes_counter.values()))):
  print(f"Frequency {frequency} class:")

  sameFreqReps = [] # create lest to keep same frequency permReps
  for record in elemsClassPerms:
    if record[2] == frequency:
      sameFreqReps.append(record[1]) #add permReps with the same frequency into list)

  print('There are ',len(sameFreqReps),' permReps there ')

  n = len(sameFreqReps)
  for i in range(n):
    for j in range(i+1,n):
      if i!=j:
        if sameFreqReps[i].isInverse(sameFreqReps[j]):
          print(f"Permutation classes {i} and {j} are related by inversion")
        if sameFreqReps[i].isBWInverse(sameFreqReps[j]):
          print(f"Permutation classes {i} and {j} are related by BW inversion")
        if sameFreqReps[i].isBWIdentical(sameFreqReps[j]):
          print(f"Permutation classes {i} and {j} are BW identical")

  print('-----')

  del sameFreqReps

Frequency 3 class:
There are  2  permReps there 
Permutation classes 0 and 1 are related by inversion
-----
Frequency 5 class:
There are  2  permReps there 
Permutation classes 0 and 1 are related by inversion
-----
Frequency 13 class:
There are  2  permReps there 
Permutation classes 0 and 1 are related by inversion
-----
Frequency 15 class:
There are  2  permReps there 
Permutation classes 0 and 1 are related by inversion
-----
Frequency 21 class:
There are  2  permReps there 
Permutation classes 0 and 1 are related by inversion
-----
Frequency 25 class:
There are  2  permReps there 
Permutation classes 0 and 1 are related by inversion
-----
Frequency 27 class:
There are  2  permReps there 
Permutation classes 0 and 1 are related by inversion
-----
Frequency 35 class:
There are  2  permReps there 
Permutation classes 0 and 1 are related by inversion
-----
Frequency 55 class:
There are  2  permReps there 
Permutation classes 0 and 1 are related by inversion
-----
Frequency 57 class:
T

In [18]:
del data, elemsClassId, elemsClassPerms, classes_counter, tempDict

D8

In [19]:
start = time.time()

# Modified files are used: all 1/2, 3/2, 5/2 are replaced by 0.5, 1.5, 2,5
with open('ADE_SimpleRootData\D8inv_SimpleRootData_mod.txt','r') as file:
    data = [ parseString(line.rstrip()) for line in file]

end = time.time()
print(end - start)

data_size = len(data)

59.282132387161255


In [20]:
# Find class of each invariant in D8 (class = invaraints with the same components)
elemsClassId = np.zeros(data_size, dtype = int)
for idx1 in tqdm(range(len(data))):
    if elemsClassId[idx1] != 0: # check if we've assigned a class to this element already
        continue
    elemsClassId[idx1] = idx1+1 # record that elem itself in its own class (shift classes number's by 1 during run for technical reasons)
    for idx2 in range(idx1+1,len(data)):
        if data[idx1][1]==data[idx2][1]: # identify components which are the same
            elemsClassId[idx2] = idx1+1 # record that elem idx2 is in the same class as element idx1 (shift classes number's by 1 during run)

elemsClassId = elemsClassId - 1 # shift classes number's back by 1

100%|██████████| 40320/40320 [00:31<00:00, 1260.86it/s] 


In [21]:
# Collect equivalent permutation for each class
elemsClassPerms = []
for classId in set(elemsClassId):
  elemsClassPerms.append([classId,PermutationRep('D8'),0]) # [Abstract id of the class, permutation class, frequency (initialised as 0)]

  for idx in range(data_size):
    if elemsClassId[idx] == classId:
      elemsClassPerms[-1][1].addPermRep(list(eval(data[idx][0])))

print(len(elemsClassPerms))

128


In [22]:
# Create a dictionary containing pairs (class index, number of elements in class)
classes_counter = dict(Counter(elemsClassId))
print('Number of classes in D8: ', len(classes_counter))

# Sort classes by the number of elements
tempDict = classes_counter
classes_counter = {k: v for k, v in sorted(tempDict.items(), key=lambda item: item[1])}

Number of classes in D8:  128


In [23]:
# add correct frequencies to every class
for idx in range(len(elemsClassPerms)):
  elemsClassPerms[idx][2] = classes_counter[elemsClassPerms[idx][0]]

In [24]:
# go over classes with the same frequencies and see which permReps are related by inversion or BW inversed duality
for frequency in sorted(list(set(classes_counter.values()))):
  print(f"Frequency {frequency} class:")

  sameFreqReps = [] # create lest to keep same frequency permReps
  for record in elemsClassPerms:
    if record[2] == frequency:
      sameFreqReps.append(record[1]) #add permReps with the same frequency into list)

  print('There are ',len(sameFreqReps),' permReps there ')

  n = len(sameFreqReps)
  for i in range(n):
    for j in range(i+1,n):
      if i!=j:
        if sameFreqReps[i].isInverse(sameFreqReps[j]):
          print(f"Permutation classes {i} and {j} are related by inversion")
        if sameFreqReps[i].isBWInverse(sameFreqReps[j]):
          print(f"Permutation classes {i} and {j} are related by BW inversion")
        if sameFreqReps[i].isBWIdentical(sameFreqReps[j]):
          print(f"Permutation classes {i} and {j} are BW identical")

        if sameFreqReps[i].is67Inverse(sameFreqReps[j]):
          print(f"Permutation classes {i} and {j} are related by 6<->7 inversion")
        if sameFreqReps[i].isBWCInverse(sameFreqReps[j]):
          print(f"Permutation classes {i} and {j} are related by BWC inversion")
        if sameFreqReps[i].isBWCIdentical(sameFreqReps[j]):
          print(f"Permutation classes {i} and {j} are BWC identical")

  print('-----')

  del sameFreqReps

Frequency 2 class:
There are  2  permReps there 
Permutation classes 0 and 1 are related by inversion
-----
Frequency 6 class:
There are  4  permReps there 
Permutation classes 0 and 2 are related by inversion
Permutation classes 0 and 3 are BW identical
Permutation classes 0 and 3 are related by 6<->7 inversion
Permutation classes 0 and 3 are BWC identical
Permutation classes 1 and 2 are BW identical
Permutation classes 1 and 2 are related by 6<->7 inversion
Permutation classes 1 and 2 are BWC identical
Permutation classes 1 and 3 are related by inversion
-----
Frequency 14 class:
There are  2  permReps there 
Permutation classes 0 and 1 are related by inversion
-----
Frequency 34 class:
There are  4  permReps there 
Permutation classes 0 and 1 are BW identical
Permutation classes 0 and 1 are related by 6<->7 inversion
Permutation classes 0 and 1 are BWC identical
Permutation classes 0 and 3 are related by inversion
Permutation classes 1 and 2 are related by inversion
Permutation clas

In [25]:
del data, elemsClassId, elemsClassPerms, classes_counter, tempDict